# Ross Ice Shelf, Antarctica

*This GNSS site has been removed by the project investigators.*

**Station Name:**  lorg

**Location:**  Lorne, Antarctica

**Archive:**  [EarthScope Consortium](http://www.earthscope.org)

**DOI:**  [https://doi.org/10.7283/ACF6-YT23](https://doi.org/10.7283/ACF6-YT23)

**Ellipsoidal Coordinates:**

- Latitude:  -78.18365
- Longitude: 170.03361
- Height:  	-7.778 m

[Station Page at EarthScope](https://www.unavco.org/instrumentation/networks/status/nota/overview/lorg)

[Station Page at Nevada Geodetic Laboratory](http://geodesy.unr.edu/NGLStationPages/stations/LORG.sta)

[Google Maps Link](https://goo.gl/maps/bSAuLXLLMmzWqPdW9) 

<img src="https://gnss-reflections.org/static/images/LORG.jpg" width="500">



### Data Summary

Station lorg is on the Ross Ice Shelf, Antarctica. The site is a largely featureless ice plain with no obstructions (see photo above). The site was installed on November 27, 2018 and decommissioned on November 15, 2019. It recorded only GPS frequencies during its operation.

LORG is an example station on the GNSSIR webapp.

Because there are no significant topographic features near the station, so it is recommended to use default values for the elevation mask. An azimuth mask is not required.

If you are curious about the position time series for the antenna, you can use `download_unr'

### Imports

In [ ]:
from gnssrefl.daily_avg_cl import daily_avg
from gnssrefl.gnssir_input import make_gnssir_input
from gnssrefl.gnssir_cl import gnssir
from gnssrefl.quickLook_cl import quicklook
from gnssrefl.rinex2snr_cl import rinex2snr
from gnssrefl.installexe_cl import installexe
import ipywidgets as widgets
import os

from pathlib import Path

from gnssrefl.utils import check_environment, set_environment, get_sys

notebook_dir = Path.cwd().parents[1]

# Making sure environment variables are set - this is required to run the gnssrefl code
exists = check_environment()
if exists == False:
    set_environment(
        refl_code=str(notebook_dir),
        orbits=str(notebook_dir / "orbits"),
        exe=str(notebook_dir / "exe"),
    )

# If you are running this locally - make sure the items in the exe folder have execution permissions

# Set local variable of refl_code location
refl_code_loc = os.environ["REFL_CODE"]
print("refl_code location:", refl_code_loc)

# import gnssrefl functions


# import the crx2rnx file which is dependent on your working OS - this is required to run the gnssrefl code
# If in docker environment, then you do not need to download crxnrnx
try:
    os.environ["DOCKER"]
except KeyError:
    sys = get_sys()
    installexe(sys)

**Fast Mode:** 
this will download data weekly instead of daily

In [ ]:
weekly = widgets.Checkbox(
    value=True, description="Fast Mode", disabled=False, indent=False
)
weekly

Deselect the Fast Mode checkbox to deactivate fast mode.

## Take a look at the Data


First, we'll make a SNR file. Let's choose day of year 205 in the year 2019 to look at.

In [ ]:
station = "lorg"
year = 2019
doy = 205

In [ ]:
rinex2snr(station, year, doy)

Use **quickLook** to produce a periodogram - quickLook is set to use the L1 frequency by default:

In [ ]:
values, metrics = quicklook(station, year, doy=doy)

This looks pretty good. It looks like we will not need an azimuth mask - which makes sense as there are no significant topographic features near the station and so we will use default values for the elevation mask and an azimuth mask is not required.

Compare the periodograms for other frequencies: L2C and L5. They should be similar to the L1 periodogram, except that there will be fewer satellite traces because only GPS satellites launched after 2005 broadcast L2C and only satellites after 2010 broadcast L5. The northwest qudarant is the noisiest and one could certainly try to improve the results by restricting some azimuths there. 

This is for L2C:

In [ ]:
values, metrics = quicklook(station, year, doy=doy, fr=20)

This is for L5:

In [ ]:
values, metrics = quicklook(station, year, doy=doy, fr=5)

### Analyze the Data

So now we prepare to analyze the data using `gnssir`.
First, we create our set of analysis instructions with `make_gnssir_input`.
The default settings only need the station name.

In [ ]:
# we saved the lat, long, and height earlier in the notebook
make_gnssir_input(station)

Next make some snr files for a time span of about eight months. We will do day of year 1 through day of year 233 for the year 2019. We can also restrict the search to the UNAVCO archive to make the
code run faster (otherwise it will check other archives as well).

In [ ]:
rinex2snr(station, year=2019, doy=1, doy_end=233, archive="unavco", weekly=weekly.value)

We would then run **gnssir** for the same time range. 

In [ ]:
gnssir(station, year=2019, doy=1, doy_end=233)

Then the **daily_avg** command will calculate the daily average reflector height from the daily output files. To minimize outliers in these daily averages, a median filter is set to allow values within a given value of the median. The user is also asked to set a required minimum number of daily satellite tracks. Here we use 0.25 meters and 50 tracks.

In [ ]:
daily_avg(station, medfilter=0.25, ReqTracks=50)

These reflector heights then give information about snow accumulation at lorg. 

We have reversed the y-axis so that the reflector height gets smaller as the snow layer increases and the distance between the antenna and snow surface decreases.